# **Introdução**

## **Objetivos:** 
### Criação de um modelo para predizer se um determinado cliente do Banco Santander irá fazer um transação ou não, independente do seu valor.

## **Dados fornecidos**
### São fornecidos dados numéricos "anonimizados", uma coluna "target" binária e uma string com a identificação "mascarada" do cliente.  
### Estão disponíveis 3 arquivos:
* Um com dados de treino
* Um com dados de teste (sem a varíavel "target" disponível)
* Um de exemplo de submissão com a predição dos dados de teste

## **Autor:** 
### André Roberto Antunes Paes

# **Análise Exploratória dos Dados**

In [ ]:
# importando bibliotecas

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Verificando os arquivos disponíveis --> código padrão Kaggle

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Obtendo os dados de treino

df = pd.read_csv('../input/santander-customer-transaction-prediction/train.csv')

df.sample(5)

In [ ]:
#  

In [ ]:
# Verificando se todos os dados são numéricos

df.info()

In [ ]:
# Confirmando que a coluna do tipo Object é a identificação do "cliente"

cols = df.dtypes.to_frame(name = 'type')
cols[cols['type'] == 'object']

In [ ]:
# Eliminando a coluna de identificação "mascarada" do cliente, já que ela não pode influenciar o modelo

df.drop(columns='ID_code', inplace=True)

In [ ]:
# Gerando uma estatística descritiva das variáveis

df.describe().T

As variáveis têm ordens de grandeza distintas. Provavelmente, uma padronização será benéfica ao modelo

In [ ]:
# Verificando se há campos nulos

df.isnull().values.any()

In [ ]:
# Avalianda a correlação linear entre as variáveis e entre elas e variável target

corr = np.array(df.corr())
np.fill_diagonal(corr, np.nan)

plt.subplots(figsize=(15, 10))
sns.heatmap(corr, cmap='seismic')

In [ ]:
# Visualmente, os pares têm baixa correlação linear. Varrendo a matriz para confirmar que não há alguma correlação forte

pairs =  np.tril_indices(corr.shape[0])
positive_pairs = corr[pairs] > 0.5 
print('Existe correlação direta forte?',np.any(positive_pairs) == True)

inverse_pairs = corr[pairs] < -0.5 
print('Existe correlação inversa forte?',np.any(inverse_pairs) == True)

# Salvando quais são as variáveis que têm alguma correlação inversa com o target

inv_cols = df.columns[corr[:,0] < 0]
inv_cols

Não há nenhuma correlação linear forte

In [ ]:
# Verificando a distrbuição das variáveis

fig, axs = plt.subplots(40,5,figsize=(25,200))
for i, col in enumerate(df.columns):
    if col != 'target': 
        j = (i - 1) // 5
        k = (i - 1) % 5
        axs[j,k].boxplot(df[col])
        axs[j,k].set_title(col)
        
plt.show()

# Verificando a realçaõ média x desvio padrão

means = df[:][1:].mean(axis=0)
stds  = df[:][1:].std(axis=0)

plt.subplots(figsize=(8, 8))
sns.scatterplot(means,stds).set( xlabel = 'Média', ylabel = 'Desvio padrão')
plt.show()

Visualmente, as variáveis aparecem bem distribuídas, de ordem de grandeza distintas. Não há nenhuma constante

In [ ]:
# Verificando o balanceamento das classes 

df.target.value_counts().to_frame(name='Target').plot(kind='bar', figsize=(10,5))
df.target.value_counts()

As classes estão desbalanceadas. Provavelmente, será preciso optar por "undersampling" ou um "oversampling"

# **Estratégia  de  validação**

A estratégia de validação será feita reservando 20% da base de treino para teste. Como vimmos que as variáveis têm
ordens de grandeza distintas, vamos padronizar a escala para que a grandeza não enviese o modelo. Como as classes estão 
desbalanceadas, será feito um balanceamento da base de treino.

In [ ]:
# Importando bibliotecas

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA

In [ ]:
# Embaralhando a base de treino, caso ela tenha vindo com alguma ordenação

df_train = shuffle(df, random_state=42)

In [ ]:
# Invertendo as correlações negativas

for col in inv_cols:
    df_train[col] = df_train[col] * -1 

In [ ]:
# Dividindo a base de treino e de teste

Y = np.array(df_train.target)

X = np.array(df_train.drop(columns=['target']))

# Código temporário com redução do tamanho da base de treino para teste de código
#sample = np.random.default_rng().integers(low=0, high=len(X), size=1000)
#X = X[sample]
#Y = Y[sample]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,shuffle=True)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

In [ ]:
# Padronizando as variáveis

#standard = preprocessing.StandardScaler()
# Testei primeiro padronizando as variáveis como planejado durante a EDA, mas obtive resultados ligeiramente melhores 
# normalizando-as
standard = preprocessing.MinMaxScaler()

X_train = standard.fit_transform(X_train)
X_test = standard.transform(X_test)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
pd.DataFrame(X_train).describe().T

In [ ]:
# Rebalanceando as classes da base de treino para que o modelo não fique tendencioso

undersample = RandomUnderSampler(sampling_strategy='majority',random_state=42)
X_train, Y_train = undersample.fit_resample(X_train, Y_train)

# Testei o balanceamento com oversampling, o tempo de processamento aumentou muito sem grandes ganhos na acurácia
#oversample = SMOTE(sampling_strategy='minority',random_state=42)
#X_train, Y_train = oversample.fit_resample(X_train, Y_train)

# Tamanhos das bases geradas

print(X_train.shape, X_train.shape, X_test.shape, Y_train.shape, Y_train.shape, Y_test.shape)


In [ ]:
# Verificando o rebalanceamento

pd.DataFrame(Y_train).value_counts().to_frame(name='Target').plot(kind='bar', figsize=(10,5))

In [ ]:
# Reduzindo a dimensionalidade

# Testei várias opções de PCA, mas nenhuma melhorou a acurácia
#pca = PCA(n_components=40, svd_solver='full')
#X_train = pca.fit_transform(X_train)
#X_test  = pca.transform(X_test)

#pca_standard = preprocessing.MinMaxScaler()
#X_train = pca_standard.fit_transform(X_train)
#X_test = pca_standard.transform(X_test)

#print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
#sns.scatterplot(X_train[:,0], X_train[:,1], hue=Y_train, palette='Set1')
#plt.show()


In [ ]:
# Após testar reduzir a dimensionalidade, testei opções com seleção de variáveis
# Selecionando as melhores variáveis de acordo com o teste de variância ANOVA ("f_classif") entre as variáveis e o target

Select = SelectKBest(f_classif, k=180)

X_train = Select.fit_transform(X_train,Y_train)
X_test  = Select.transform(X_test)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

# **Treino do modelo**

Foram treinados e avaliados vários modelos com hiperparâmetros distintos para encontrar o ideal. Com a base de treino fornecida, um algoritmo "burro" que definisse 0 para toda a saída, teria, matematicamente, uma acurácia de ~90%, já que as classes estão bem desbalanceadas. 

In [ ]:
# Importando bibliotecas

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

import random as rd
rd.seed(42)

In [ ]:
# Definido função para treinar e imprimir resultados de um modelo

def train_results(algorithm, model, parameters, X_train, Y_train, X_test, Y_test):
    
    model = GridSearchCV(model, parameters, scoring = 'roc_auc', cv=5, verbose=1, return_train_score=True, n_jobs=-1)

    model.fit(X_train, Y_train)
    print('Melhores hyperparâmetros para ' + algorithm + ':', model.best_params_)
    
    yhat       = model.predict(X_train)
    yhat_proba = model.predict_proba(X_train)[:,1]
    print('---------------------- Desempenho de treino ----------------------')
    print_results(Y_train, yhat, yhat_proba)
    
    yhat       = model.predict(X_test)
    yhat_proba = model.predict_proba(X_test)[:,1]
    print('---------------------- Desempenho de teste ----------------------')
    print_results(Y_test, yhat, yhat_proba)
    
    return model.best_estimator_


# Função para imprimir resultados

def print_results(y_actual,y_predicted, y_probable):
    
    cf_matrix = confusion_matrix(y_actual,y_predicted)
    sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.0%', cmap='seismic')
    plt.show()
    print(classification_report(y_actual,y_predicted))
    print('**** AUC =', roc_auc_score(y_actual,y_probable))
    

In [ ]:
# Treinando e encontrando os melhores hyperparâmetros para regressão logística

from sklearn.linear_model import LogisticRegression

parameters ={'penalty':['l2'],'dual':[False], 'tol':[0.0001],'C':[0.01],'fit_intercept':[True],'intercept_scaling':[1],'class_weight':['balanced'],
             'random_state':[42],'solver':['newton-cg','lbfgs'],'max_iter':[5000],'multi_class':['auto'],'verbose':[0],'warm_start':[False],
             'n_jobs':[-1],'l1_ratio':[None]}
#'class_weight':[{0:.1,1:.9}]

logreg = LogisticRegression()

logreg = train_results('Regressão Logística',logreg,parameters,X_train,Y_train,X_test,Y_test)

In [ ]:
# Treinando e encontrando os melhores hyperparâmetros para Support Vector Machine

#from sklearn.svm import LinearSVC
from sklearn.svm import SVC

#parameters ={'penalty':['l2'],'loss':['squared_hinge'],'dual':[False],'tol':[0.000001],'C':[1.0],
#             'fit_intercept':[True],'intercept_scaling':[1],'class_weight':[None], 
#             'verbose':[0],'random_state':[42],'max_iter':[10000]}

parameters ={'kernel':['linear','rbf'],'C':[0.1],'degree':[3],'gamma':['scale'],'coef0':[0.0],
             'shrinking':[False],'probability':[False],'tol':[0.001],'cache_size':[200], 
             'class_weight':[None],'verbose':[0],'max_iter':[5000],'break_ties':[False],
             'random_state':[42],'probability':[True]}
#'class_weight':[None,{0:.15,1:.85}
             
#supvec = LinearSVC()
supvec = SVC()

supvec = train_results('Linear SVM',supvec,parameters,X_train,Y_train,X_test,Y_test)

In [ ]:
### Treinando e encontrando os melhores hyperparâmetros para random forest

from sklearn.ensemble import RandomForestClassifier

parameters ={'n_estimators':[100,200],'criterion':['gini','entropy'],'max_depth':[None],'min_samples_split':[1000],
             'min_samples_leaf':[1000],'min_weight_fraction_leaf':[0.0],'max_features':['sqrt'],'max_leaf_nodes':[None],
             'min_impurity_decrease':[0.0],'bootstrap':[True],'oob_score':[False],'n_jobs':[-1],'random_state':[42],
             'verbose':[0],'warm_start':[False],'class_weight':[None],'ccp_alpha':[0.0],'max_samples':[None]}
#'class_weight':['balanced',{0:.15,1:.85}]

ranfor = RandomForestClassifier()

ranfor = train_results('Random Forest', ranfor,parameters, X_train,Y_train,X_test,Y_test)

In [ ]:
# Treinando e encontrando os melhores hyperparâmetros para um algoritmo composto

from sklearn.ensemble import HistGradientBoostingClassifier

parameters ={'loss':['auto'],'learning_rate':[0.05,0.1],'max_iter':[10000], 'max_leaf_nodes':[None], 
             'max_depth':[None],'min_samples_leaf':[1000],'l2_regularization':[0.1],'max_bins':[255], 
             'categorical_features':[None],'monotonic_cst':[None],'warm_start':[False],
             'early_stopping':['auto'],'scoring':['roc_auc'],'validation_fraction':[0.1],'n_iter_no_change':[100],
             'tol':[1e-07],'random_state':[42],'verbose':[0]}

graboos=HistGradientBoostingClassifier()

graboos = train_results('Gradient Boosting', graboos,parameters, X_train,Y_train,X_test,Y_test)

In [ ]:
#  Como o balanceamento, a redução de dimensionalidade, a seleção de features e o ajuste de diferentes hiperparâmetros em 
# diferentes algoritmos geraram resultados (AUC ~0.80) aquém da meta (AUC 0.85), usando o PCA para, 
# ao invés de reduzir a dimensionalidade, adicionar novas variáveis

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1,random_state=42,shuffle=True)

X_train = standard.fit_transform(X_train)
X_test = standard.transform(X_test)

undersample = RandomUnderSampler(sampling_strategy='majority',random_state=42)
X_train, Y_train = undersample.fit_resample(X_train, Y_train)

pca = PCA(n_components=20, svd_solver='full')
X_train_new_cols = pca.fit_transform(X_train)
X_test_new_cols  = pca.transform(X_test)

pca_standard = preprocessing.MinMaxScaler()
X_train_new_cols = pca_standard.fit_transform(X_train_new_cols)
X_test_new_cols  = pca_standard.transform(X_test_new_cols)

new_X_train = np.array(pd.concat([pd.DataFrame(X_train), pd.DataFrame(X_train_new_cols)], axis = 1))
new_X_test = np.array(pd.concat([pd.DataFrame(X_test), pd.DataFrame(X_test_new_cols)], axis = 1))

print(new_X_train.shape, new_X_test.shape, Y_train.shape, Y_test.shape)

In [ ]:
# Reavaliando os hiperparâmetros com as novas variáveis

from sklearn.ensemble import HistGradientBoostingClassifier

parameters ={'loss':['auto'],'learning_rate':[0.04,0.05],'max_iter':[10000], 'max_leaf_nodes':[None], 
             'max_depth':[None],'min_samples_leaf':[1000],'l2_regularization':[0.1,0.5],'max_bins':[255], 
             'categorical_features':[None],'monotonic_cst':[None],'warm_start':[False],
             'early_stopping':['auto'],'scoring':['roc_auc'],'validation_fraction':[0.1],'n_iter_no_change':[100],
             'tol':[1e-07],'random_state':[42],'verbose':[0]}

graboos=HistGradientBoostingClassifier()

graboos = train_results('Gradient Boosting', graboos,parameters, new_X_train,Y_train,new_X_test,Y_test)

In [ ]:
# Criando um dataframe com os dados das taxas de positivo e falso positivo por "threshold" para tentar encontrar 
# um ponto "ótimo" onde a classe 1 seja considerada no modelo de melhor desempenho para a base de teste

from sklearn.metrics import roc_curve

probabilites = graboos.predict_proba(new_X_test)[:,1]

false_pos_rate, tru_pos_rate, thresholds = roc_curve(Y_test, probabilites)
tru_pos_rate = 1 - tru_pos_rate

thresholds = pd.DataFrame({'Threshold':thresholds,'Falso Positivo':false_pos_rate,'Verd. Positivo': tru_pos_rate})
thresholds.head()

In [ ]:
# Gerando o gráfico para escolher um threshold a partir do qual consideraremos a classe 1

thresholds.plot(x='Threshold',y=['Verd. Positivo','Falso Positivo'], figsize=(10,5))

In [ ]:
# Procurando o threshold que fornece o melhor resultado

best_auc = 0.88
best_threshold = 0

for threshold in range(5000,10000,1):
    #new_auc = roc_auc_score(Y_test, np.where(probabilites >= threshold / 10000, 1, 0))
    new_probabilities = np.copy(probabilites)
    chg_probabilites = np.where(new_probabilities >= threshold / 10000)
    new_probabilities[chg_probabilites] = 1
    yhat = new_probabilities
    new_auc = roc_auc_score(Y_test,yhat)
    if new_auc > best_auc:
        #print(best_auc , new_auc)
        best_auc = new_auc
        best_threshold = threshold  

best_threshold = best_threshold / 10000

print('Melhor threshold:', best_threshold)

print('---------------------- Desempenho de teste com ajuste do threshold ----------------------')
print('**** AUC =', best_auc)

Como os resultados não melhoraram muito, treinando um modelo com o lightgbm

In [ ]:
# Validação cruzada com o lightgbm

import lightgbm

Y = df.target
X = df.drop(columns=['target'])

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

train_data = lightgbm.Dataset(X_train, label=Y_train)

parameters = {'objective': 'binary','metric': 'auc','is_unbalance': 'true','boosting': 'gbdt','num_leaves': 63,
              'feature_fraction': 0.5,'bagging_fraction': 0.5,'bagging_freq': 20,'learning_rate': 0.01,
              'verbose':0,'force_col_wise':[True]}

eval_hist = lightgbm.cv(parameters, train_data, num_boost_round=5000, nfold=5, stratified=True, 
                         shuffle=True, verbose_eval = 50)

eval_hist.keys()

In [ ]:
# Treinando o modelo Lightgbm

best_round =  np.argmax(eval_hist['auc-mean'])
print('qtde de "rounds":', best_round )

model_lgbm = lightgbm.train(parameters,train_data,num_boost_round=best_round)

yhat = model_lgbm.predict(X_train)
print('---------------------- Desempenho de treino ----------------------')
print('**** AUC =', roc_auc_score(Y_train,yhat))

yhat = model_lgbm.predict(X_test)
print('---------------------- Desempenho de teste ----------------------')
print('**** AUC =', roc_auc_score(Y_test,yhat))

# **Submissão**
Execução da predição com o modelo final nos dados de teste e geração de um arquivo de submissão no formato da competição.

In [ ]:
# Retreinando o modelo com todos os dados

train_data = lightgbm.Dataset(X, label=Y)

model_lgbm = lightgbm.train(parameters,train_data,num_boost_round=best_round)

yhat = model_lgbm.predict(X)
print('---------------------- Desempenho de treino final ----------------------')
print('**** AUC =', roc_auc_score(Y,yhat))

In [ ]:
# recuperando o arquivo com as variáveis para teste

test = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')

test.head()

In [ ]:
# Verificando se ele não tem nulos

test.isnull().values.any()

In [ ]:
# Dando uma olhada em como é o arquivo de exemplo de submissão

example = pd.read_csv('../input/santander-customer-transaction-prediction/sample_submission.csv')

example.head()

In [ ]:
# fazendo as predições

X = test.drop(columns=['ID_code'])
    
target = model_lgbm.predict(X)

In [ ]:
target

In [ ]:
# Criando o arquivo de submissão

submission = test[['ID_code']]

submission['target'] = pd.Series(target)

submission.head()

In [ ]:
# Gravando arquivo de saída

submission.to_csv('submission.csv',index=False)

Referência: LGBM with parameters (https://www.kaggle.com/code/dimanjung/lgbm-with-parameters)